# 🚀 Streamlit Launcher per la Pipeline Manga-to-Anime

Questo notebook prepara l'ambiente Colab e lancia l'applicazione Streamlit tramite `ngrok` per renderla accessibile pubblicamente.

In [ ]:
# Step 1: Clona il tuo repository GitHub
# Assicurati che l'URL sia corretto e che il repository sia pubblico.
!git clone https://github.com/maurocarlu/ToonCrafter.git
%cd ToonCrafter


In [ ]:
# Step 2: Installa tutte le dipendenze dal file requirements.txt
# Questo processo può richiedere alcuni minuti.
!pip install -r requirements.txt


In [ ]:
# Aggiungi in una cella/entrypoint prima di importare scikit-video o librerie legacy
import numpy as _np
if not hasattr(_np, "float"):
    _np.float = float
    _np.int = int
    _np.complex = complex
    _np.bool = bool
    _np.object = object

In [ ]:
# Step 3: Avvia Streamlit con ngrok (Colab)
import os, time, subprocess, requests, sys
from pyngrok import ngrok

# (Opzionale) Incolla il tuo token ngrok per sessioni più stabili
NGROK_AUTH_TOKEN = "32TyeKBnn1AXBPbExme5USEWkFi_eh9PZv8UXAbTXhiG2mAr"  # <-- TOKEN (opzionale)

# Chiudi eventuali tunnel/processi precedenti
try:
    ngrok.kill()
except Exception:
    pass

# Imposta token se presente
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Avvia Streamlit in modalità headless su 0.0.0.0:8501
env = os.environ.copy()
cmd = [
    sys.executable, "-m", "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.address", "0.0.0.0",
    "--server.headless", "true",
    "--browser.gatherUsageStats", "false",
]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Attendi che Streamlit sia pronto
print("Avvio di Streamlit in corso...")
ready = False
for _ in range(60):
    try:
        r = requests.get("http://127.0.0.1:8501/healthz", timeout=2)
        if r.ok:
            ready = True
            break
    except Exception:
        pass
    time.sleep(1)

if not ready:
    print("⚠️ Streamlit non ha risposto in tempo. Mostro gli ultimi log:")
    try:
        # stampa qualcosa dai log
        for _ in range(50):
            line = proc.stdout.readline()
            if not line:
                break
            print(line, end="")
    except Exception:
        pass

print("Creazione del tunnel ngrok...")
tunnel = ngrok.connect(addr=8501, proto="http", bind_tls=True)
public_url = tunnel.public_url
print("\n" + "="*70)
print("🚀 LA TUA APP È ONLINE (ngrok):")
print(public_url)
print("="*70)
print("Lascia in esecuzione questa cella. Per fermare: Interrompi esecuzione.")

# Mantieni la cella viva finché il processo è attivo
try:
    while proc.poll() is None:
        time.sleep(5)
except KeyboardInterrupt:
    print("Interruzione richiesta. Chiudo ngrok e Streamlit...")
finally:
    try:
        ngrok.kill()
    except Exception:
        pass
    try:
        proc.terminate()
    except Exception:
        pass